In [1]:
import logging
import warnings
logging.getLogger('fbprophet').setLevel(logging.ERROR)
logging.getLogger('statsmodels').setLevel(logging.WARNING)
logging.getLogger('arch').setLevel(logging.WARNING)
logging.getLogger('croston').setLevel(logging.WARNING)
logging.getLogger('pandas').setLevel(logging.WARNING)
logging.getLogger('pmdarima').setLevel(logging.WARNING)
logging.getLogger('causalimpact').setLevel(logging.WARNING)
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [2]:
import forecast_engine.modelling.utils.nodes as utils
import forecast_engine.modelling.forecast_engine.forecast_loop as forecast_loop
import forecast_engine.modelling.forecast_engine.forecast_engine_pipeline as forecast_engine_pipeline





import forecast_engine.modelling.models.ml_models.naive_models as naive_models
import forecast_engine.modelling.models.ml_models.holt_winters as holt_winters

import forecast_engine.modelling.models.ml_models.garch as garch
import forecast_engine.modelling.models.ml_models.rw_drift as rw_drift
import forecast_engine.modelling.models.ml_models.arima as arima
import forecast_engine.modelling.models.ml_models.croston as croston

import forecast_engine.modelling.models.ml_models.causal_impact as causal_impact
import forecast_engine.modelling.models.ml_models.prophet as prophet

import forecast_engine.modelling.models.ml_models.light_gbm as light_gbm
import forecast_engine.modelling.models.ml_models.xgboost as xgboost




import forecast_engine.modelling.models.ml_models.linear_regression_mv as linear_regression_mv
import forecast_engine.modelling.models.ml_models.arima_mv as arima_mv
import forecast_engine.modelling.models.ml_models.causal_impact_mv as causal_impact_mv
import forecast_engine.modelling.models.ml_models.xgboost_mv as xgboost_mv
import forecast_engine.modelling.models.ml_models.light_gbm_mv as light_gbm_mv
import forecast_engine.modelling.models.ml_models.prophet_mv as prophet_mv
import forecast_engine.modelling.models.ml_models.olsregression_mv as olsregression_mv




import pandas as pd
import numpy as np
import datetime


PREPROCESS

In [3]:
import forecast_engine.preprocess.nodes as preprocess

SALES ORDER

In [4]:
path = r"/Users/anand_k/Downloads/UPL/Demand forecasting datasets [DO NOT EDIT]/Sales order"
sales_data, df = preprocess.read_prep_sales_order(path)
dates, act_dates = preprocess.prep_dates_list(df)

In [5]:
sub_season = [str(x)+'_Season' for x in dates]
df[sub_season] = df.apply( lambda x : preprocess.decomposition(x,dates,df.columns), axis = 1, result_type="expand")

In [6]:
df['intermittence'] = df.apply(lambda x : preprocess.intermittence(x), axis =1)
df['cov'] = df[dates].apply(lambda x : preprocess.cov(x), axis =1)
df['seq - flag'] = df.apply(lambda x : preprocess.seq_flag(x,dates,df.columns) , axis = 1)

In [7]:
df.drop(['intermittence','cov'],axis =1,inplace=True)

WEATHER DATA

In [8]:
path = r"/Users/anand_k/Downloads/UPL/Demand forecasting datasets [DO NOT EDIT]/Weather Data/Weather Data.xlsx"
wthr_data = preprocess.read_prep_weather(path)

In [9]:
wthr_data

,Country,2019 - 01_Avg Temp (C),2019 - 02_Avg Temp (C),2019 - 03_Avg Temp (C),2019 - 04_Avg Temp (C),2019 - 05_Avg Temp (C),2019 - 06_Avg Temp (C),2019 - 07_Avg Temp (C),2019 - 08_Avg Temp (C),2019 - 09_Avg Temp (C),...,2021 - 10_Relative Humidity (%),2021 - 11_Relative Humidity (%),2021 - 12_Relative Humidity (%),2022 - 01_Relative Humidity (%),2022 - 02_Relative Humidity (%),2022 - 03_Relative Humidity (%),2022 - 04_Relative Humidity (%),2022 - 05_Relative Humidity (%),2022 - 06_Relative Humidity (%),2022 - 07_Relative Humidity (%)
0,Mexico,15.100685,17.501682,18.93809,20.299129,22.072156,21.325376,20.17738,20.719087,19.970685,...,74.688081,67.689772,61.966188,63.661877,58.495351,49.211327,53.776839,52.436179,68.25951,72.98732


MACROECONOMIC DATA

In [10]:
path = r"/Users/anand_k/Downloads/UPL/Demand forecasting datasets [DO NOT EDIT]/Macro-economics"
macro = preprocess.read_macro_files(path, country = ['Mexico'])

In [11]:
macro

,2019 - 01_Inflation Data,2019 - 02_Inflation Data,2019 - 03_Inflation Data,2019 - 04_Inflation Data,2019 - 05_Inflation Data,2019 - 06_Inflation Data,2019 - 07_Inflation Data,2019 - 08_Inflation Data,2019 - 09_Inflation Data,2019 - 10_Inflation Data,...,2022 - 04_Lending Rate,2022 - 05_Lending Rate,2022 - 06_Lending Rate,2022 - 07_Lending Rate,2022 - 08_Lending Rate,2022 - 09_Lending Rate,2022 - 10_Lending Rate,2022 - 11_Lending Rate,2022 - 12_Lending Rate,Country
0,0.0437,0.0394,0.04,0.0441,0.0428,0.0395,0.0378,0.0316,0.03,0.0302,...,0.065,0.07,0.0775,0.0775,0.085,0.0925,0.0925,0.1,0.105,Mexico


MERGE DATA

In [12]:
df['Country'] = 'Mexico'
data = pd.merge(df, macro, on = 'Country', how = 'left')
data = pd.merge(data, wthr_data, on = 'Country', how = 'left')
data.drop('Country',axis=1,inplace=True)
data.rename(columns={'Material':'index'},inplace=True)
data.columns = [str(i) for i in list(data.columns) ]
    

In [13]:
data

,index,2019 - 03,2019 - 04,2019 - 05,2019 - 06,2019 - 07,2019 - 08,2019 - 09,2019 - 10,2019 - 11,...,2021 - 10_Relative Humidity (%),2021 - 11_Relative Humidity (%),2021 - 12_Relative Humidity (%),2022 - 01_Relative Humidity (%),2022 - 02_Relative Humidity (%),2022 - 03_Relative Humidity (%),2022 - 04_Relative Humidity (%),2022 - 05_Relative Humidity (%),2022 - 06_Relative Humidity (%),2022 - 07_Relative Humidity (%)
0,U00000000000180123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,NaN,...,74.688081,67.689772,61.966188,63.661877,58.495351,49.211327,53.776839,52.436179,68.25951,72.98732
1,U00000000000208289,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,74.688081,67.689772,61.966188,63.661877,58.495351,49.211327,53.776839,52.436179,68.25951,72.98732
2,U00000000000368011,NaN,768.0,1044.0,1824.0,1128.0,1428.0,1824.0,1228.0,564.0,...,74.688081,67.689772,61.966188,63.661877,58.495351,49.211327,53.776839,52.436179,68.25951,72.98732
3,U00000000000554501,NaN,10000.0,19250.0,5000.0,NaN,NaN,NaN,3500.0,NaN,...,74.688081,67.689772,61.966188,63.661877,58.495351,49.211327,53.776839,52.436179,68.25951,72.98732
4,U00000000000563501,NaN,600.0,4300.0,2350.0,NaN,NaN,NaN,4450.0,2000.0,...,74.688081,67.689772,61.966188,63.661877,58.495351,49.211327,53.776839,52.436179,68.25951,72.98732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216,U00000000189060238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,74.688081,67.689772,61.966188,63.661877,58.495351,49.211327,53.776839,52.436179,68.25951,72.98732
1217,U00000000189060244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,74.688081,67.689772,61.966188,63.661877,58.495351,49.211327,53.776839,52.436179,68.25951,72.98732
1218,U00000000189060261,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,74.688081,67.689772,61.966188,63.661877,58.495351,49.211327,53.776839,52.436179,68.25951,72.98732
1219,U00000000189060268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,74.688081,67.689772,61.966188,63.661877,58.495351,49.211327,53.776839,52.436179,68.25951,72.98732


Forecast Engine

In [14]:
#Flags
debug_flag = 0
debug_models = []
outlier_flag = 1
select_variable_flag = 0
all_at_once_flag = 1
months_to_forecast = 3
max_validation = 12
validation_window = 4
single_series = 0
multivariate_flag = 1

univariate_models = {'hw_ets_r_MNN':holt_winters.hw_ets_r_MNN, 'hw_ets_ATA':holt_winters.hw_ets_ATA, 'hw_ets_AFA':holt_winters.hw_ets_AFA, 
                    'hw_ets_ATN':holt_winters.hw_ets_ATN, 'hw_ets_NFA':holt_winters.hw_ets_NFA, 'hw_ets_NFN':holt_winters.hw_ets_NFN, 'hw_ets_AFN':holt_winters.hw_ets_AFN,
                    'hw_ets_r_MAA':holt_winters.hw_ets_r_MAA,'hw_ets_r_MAN':holt_winters.hw_ets_r_MAN, 'hw_ets_r_MNA':holt_winters.hw_ets_r_MNA,
                    
                    'mean_model':naive_models.mean_model, 'median_model':naive_models.median_model, 'naive':naive_models.naive, 'naive_ets':naive_models.naive_ets,
                    'snaive':naive_models.snaive, 'snaive_twoseasons':naive_models.snaive_twoseasons,
                    
                    'rw_drift':rw_drift.rw_drift, 'croston':croston.croston, 'arima':arima.arima, 'garch':garch.garch,

                    'xgboost':xgboost.xgboost,'prophet':prophet.prophet,'causalimpact':causal_impact.causalimpact,'light_gbm':light_gbm.lgbm,'linearregression_mv':linear_regression_mv.linearregression_mv}


multivariate_models = {'linearregression_mv':linear_regression_mv.linearregression_mv, 'arima_mv':arima_mv.arima_mv, 
                        'xgboost_mv':xgboost_mv.xgboost_mv, 'prophet_mv':prophet_mv.prophet_mv, 'olsregression_mv':olsregression_mv.olsregression_mv}
                        #TODO 'light_gbm_mv':light_gbm_mv.lgbm_mv, 'causalimpact_mv':causal_impact_mv.causalimpact_mv, random_forecast, random_forest_mv
                    

list_of_models = univariate_models.copy()
if multivariate_flag == 1:  list_of_models.update(multivariate_models)


In [15]:
df_output = data.head(10).apply(lambda x: forecast_engine_pipeline.forecast_engine(x, months_to_forecast,debug_flag,debug_models,list_of_models, single_series,  all_at_once_flag, validation_window, max_validation, outlier_flag, select_variable_flag),axis = 1)    

U00000000000180123
       Header   Value  Avg Temp (C) Inflation Data Lending Rate  Max Temp (C)  \
40  2022 - 05  2725.0     22.117168         0.0765         0.07     32.208969   
41  2022 - 06  2725.0     20.450507         0.0799       0.0775     30.314979   
42  2022 - 07  2725.0     20.463593         0.0815       0.0775     29.184962   
43  2022 - 08  2725.0     20.463593          0.087        0.085     29.184962   
44  2022 - 09  2725.0     20.463593          0.087       0.0925     29.184962   
45  2022 - 10  2725.0     20.463593         0.0841       0.0925     29.184962   
46  2022 - 11  2725.0     20.463593          0.078          0.1     29.184962   
47  2022 - 12  2725.0     20.463593         0.0782        0.105     29.184962   
48  2023 - 01     NaN           NaN            NaN          NaN           NaN   
49  2023 - 02     NaN           NaN            NaN          NaN           NaN   
50  2023 - 03     NaN           NaN            NaN          NaN           NaN   
51  2023 

/Users/anand_k/Downloads/UPL/Forecast Engine/venv/lib/python3.9/site-packages/statsmodels/tsa/holtwinters/model.py:915: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


Optimization not converged


 This problem is unconstrained.
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  9.14648D-01    |proj g|=  1.13971D+01

At iterate    5    f= -8.44184D-02    |proj g|=  2.33229D+00

At iterate   10    f= -1.31198D-01    |proj g|=  1.96073D-01

At iterate   15    f= -1.57646D-01    |proj g|=  6.14485D-02

At iterate   20    f= -1.65171D-01    |proj g|=  2.72148D-01

At iterate   25    f= -1.66504D-01    |proj g|=  1.24778D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4     27     35      1     0     0   

...

/Users/anand_k/Downloads/UPL/Forecast Engine/venv/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
 This problem is unconstrained.



At iterate   10    f=  3.69461D+00    |proj g|=  1.80142D-02

At iterate   15    f=  3.69425D+00    |proj g|=  1.92404D-02

At iterate   20    f=  3.68802D+00    |proj g|=  5.59051D-02

At iterate   25    f=  3.68343D+00    |proj g|=  7.31818D-03

At iterate   30    f=  3.68318D+00    |proj g|=  1.36181D-04

At iterate   35    f=  3.68318D+00    |proj g|=  1.15898D-04

At iterate   40    f=  3.68318D+00    |proj g|=  1.43999D-04

At iterate   45    f=  3.68318D+00    |proj g|=  1.54282D-03

At iterate   50    f=  3.68241D+00    |proj g|=  2.23217D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   16     50     57   

/Users/anand_k/Downloads/UPL/Forecast Engine/venv/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
 This problem is unconstrained.



At iterate   10    f=  4.21421D+00    |proj g|=  2.46914D-03

At iterate   15    f=  4.21370D+00    |proj g|=  3.07224D-02

At iterate   20    f=  4.20357D+00    |proj g|=  1.11100D-01

At iterate   25    f=  4.20215D+00    |proj g|=  6.07303D-03

At iterate   30    f=  4.20188D+00    |proj g|=  3.14065D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   16     34     42      1     0     0   4.245D-04   4.202D+00
  F =   4.2018813750030040     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
xgboost_mv : errored_out
prophet_mv : errored_out
olsregression_mv : errored_out
               models      smape

<Figure size 640x480 with 0 Axes>

In [18]:
df_output = df_output.ravel()
output_temp = df_output[0]['output']
df_classifier_train_set = df_output[0]['classifier_train_set']
impact_temp = df_output[0]['var_coeff']
df_univariate_temp = df_output[0]['df_univariate']
mp = df_output[0]['df_multivariate']
df_ensemble_temp = df_output[0]['df_ensemble']
df_fitted_values_list = df_output[0]['fitted_values_list']
model_list = df_output[0]['dataf']
output_temp = df_output.ravel()[0]['output']

DEBUG THE ENGINE

In [ ]:
# # Function test
# ti = data[data['index']=='U00000000006030126'].squeeze(axis=0)
# from pandas import MultiIndex
# import datetime 
# single_series = 0
# ts1 = datetime.datetime.now().timestamp()
# if (single_series == 0):
#   ti = pd.Series(ti)
# series_key = str(ti['index'])
# print(series_key)
# ti1, ti_key, variable_list, series_flag = utils.get_ti_in_shape(ti.copy())
# ti = ti1.copy()
# ts2 = datetime.datetime.now().timestamp()
# ti1 = utils.driver_imputation(ti.copy())
# ti=ti1.copy()
# ti = ti[2:]
  
# forecast_loop.forecast_loop_function(ti, series_flag, months_to_forecast, debug_flag, debug_models, list_of_models, variable_list, single_series, all_at_once_flag, validation_window, max_validation, outlier_flag, select_variable_flag)